# Set Up

In [4]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import folium
import random
from scipy.signal import sepfir2d
import math
%matplotlib inline

from os import path, getcwd
from glob import glob

code_dir = getcwd()
data_dir = path.expanduser('~/data/workshop-content18/3-snc/data/')

ais_pathnames = glob(data_dir + '*.txt')
ais_basenames = [path.basename(pn) for pn in ais_pathnames]

delta_cur_basename = next(bn for bn in ais_basenames if 'Deltaport_Current' in bn)
delta_his_basename = next(bn for bn in ais_basenames if 'Deltaport_History' in bn)
nwest_cur_basename = next(bn for bn in ais_basenames if 'NewWestminster_Current' in bn)
nwest_his_basename = next(bn for bn in ais_basenames if 'NewWestminster_History' in bn)

max_rows=int(3e6)
delta_cur = pd.read_csv(
    data_dir + delta_cur_basename, sep='\t', nrows=max_rows, low_memory=False,
    parse_dates=['ReceivedTime'])

delta_his = pd.read_csv(
    data_dir + delta_his_basename, sep='\t', nrows=max_rows, low_memory=False,
    parse_dates=['ReceivedTime'])

nwest_cur = pd.read_csv(
    data_dir + nwest_cur_basename, sep='\t', nrows=max_rows, low_memory=False,
    parse_dates=['ReceivedTime'])

nwest_his = pd.read_csv(
    data_dir + nwest_his_basename, sep='\t', nrows=max_rows, low_memory=False,
    parse_dates=['ReceivedTime'])

delta_cur = pd.concat([delta_cur, delta_his, nwest_cur, nwest_his])
delta_cur = delta_cur.dropna()

delta_his = None
nwest_cur = None
nwest_his = None

# One Ship

In [5]:
def RGBToHTMLColor(rgb_tuple):
    """ convert an (R, G, B) tuple to #RRGGBB """
    hexcolor = '#%02x%02x%02x' % rgb_tuple
    # that's it! '%02x' means zero-padded, 2-digit hex values
    return hexcolor

In [6]:
#i = random.randint(1,int(UserID_vc.size))
# 117 -- nice one
# 95 -- crosses land like crazy, also swims in the river
#i = 95
i = 272

UserID_vc = pd.value_counts(delta_cur.UserID)
ship = (delta_cur.loc[delta_cur.UserID == UserID_vc.index[i]])
#ship.plot.scatter('Longitude', 'Latitude', title='Index: ' + str(i));

ship_sparse = (ship.reset_index(drop=True)
                   .iloc[::10, :]
                   .reset_index(drop=True))
m = folium.Map(location=[49, -123.7], 
               tiles='Stamen Terrain', zoom_start=9)
latlon = zip(list(ship_sparse.Latitude.values), 
             list(ship_sparse.Longitude.values))
folium.PolyLine(list(latlon), color='red').add_to(m)
j = 1
N = len(ship_sparse.Latitude.values)
#for coord in latlon:
#    folium.Circle(location=coord, popup=str(j), fill=True, color='blue').add_to(m)
    #folium.Circle(location=coord, popup=str(j), fill=True,
    #              color=RGBToHTMLColor(((2*j)%255,(5*j)%255,(10*j)%255))).add_to(m)
#    j += 1

m